In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import keras.backend as K
from keras.engine import Layer

from keras.models import Sequential,Model
from keras.layers import Input, Dropout, merge
from keras.layers.convolutional import Convolution2D, UpSampling2D, ZeroPadding2D, Cropping2D, Deconvolution2D
from keras.layers.core import Activation
from keras.optimizers import Adam, SGD

from keras.applications.resnet50 import ResNet50
from keras.models import Model, load_model
from keras.metrics import binary_accuracy

import numpy as np
import tensorflow as tf



In [4]:
#Model parameters
stride = 32
learning_rate = 0.1
N_CLASSES = 2
batch_size = 5
N_epochs = 100
IMG_SIZE = 128

# Init the VGG model
vgg_conv = tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
for layer in vgg_conv.layers[:]:
    layer.trainable = False

# ENCODER
# Create the model based /on VGG16
model_with_vgg = Sequential()
model_with_vgg.add(vgg_conv)

#DECODER
# Upsampling
model_with_vgg.add(Deconvolution2D(N_CLASSES, (16, 16), strides=4, padding='same') )
model_with_vgg.add(Dropout(0.2))
model_with_vgg.add(Deconvolution2D(N_CLASSES, (32, 32), strides=4, padding='same') )
model_with_vgg.add(Dropout(0.2))
model_with_vgg.add(Deconvolution2D(N_CLASSES, (64, 64), strides=2, padding='same') )

model_with_vgg.add(Activation('softmax'))
model_with_vgg.summary()        


opt = tf.keras.layers.LeakyReLU()
m = tf.keras.metrics.MeanIoU(num_classes=2)

def dice(y_true, y_pred):
    # Symbolically compute the intersection
    y_int = y_true*y_pred
    # Technically this is the negative of the Sorensen-Dice index. This is done for
    # minimization purposes
    return -(2*K.sum(y_int) / (K.sum(y_true) + K.sum(y_pred)))

model_with_vgg.compile(loss="binary_crossentropy",
              optimizer="Adam",
              metrics=['accuracy', m, dice])

80142336/80134624 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 4, 4, 512)         20024384  
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 16, 16, 2)         262146    
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 2)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 64, 64, 2)         4098      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 2)         0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 128, 128, 2)       16386     
_________________________________________________________

In [ ]:
model_with_vgg.fit(
    X_train, new_Y,
    validation_split=0.1,
    batch_size = batch_size,
    epochs = 300
)

In [15]:
#Model parameters
stride = 32
learning_rate = 0.1
N_CLASSES = 2
batch_size = 5
N_epochs = 100

# Init the ResNet50 model
resnet_conv = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
for layer in resnet_conv.layers[:]:
    layer.trainable = False

# ENCODER
model_with_resnet = Sequential()
model_with_resnet.add(resnet_conv)

#model_with_resnet.add(Convolution2D(32,kernel_size=(1,1),padding = "same",activation = "relu",name = "fc6"))
#model_with_resnet.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

# add classifier - to get size down to at most 8x8
model_with_resnet.add(Deconvolution2D(N_CLASSES, (4, 4), strides=2, padding='same') )
model_with_resnet.add(Dropout(0.2))
model_with_resnet.add(Deconvolution2D(N_CLASSES, (16, 16), strides=2, padding='same') )
model_with_resnet.add(Dropout(0.2))
model_with_resnet.add(Deconvolution2D(N_CLASSES, (32, 32), strides=2, padding='same') )
model_with_resnet.add(Dropout(0.2))
model_with_resnet.add(Deconvolution2D(N_CLASSES, (32, 32), strides=2, padding='same') )
model_with_resnet.add(Dropout(0.2))
model_with_resnet.add(Deconvolution2D(N_CLASSES, (64, 64), strides=2, padding='same') )

model_with_resnet.add(Activation('softmax'))
model_with_resnet.summary()        

opt = tf.keras.layers.LeakyReLU()
m = tf.keras.metrics.MeanIoU(num_classes=2)

def dice(y_true, y_pred):
    # Symbolically compute the intersection
    y_int = y_true*y_pred
    # Technically this is the negative of the Sorensen-Dice index. This is done for
    # minimization purposes
    return -(2*K.sum(y_int) / (K.sum(y_true) + K.sum(y_pred)))

model_with_resnet.compile(loss="binary_crossentropy",
              optimizer="Adam",
              metrics=['accuracy', m, dice])

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 4, 4, 2048)        23587712  
_________________________________________________________________
conv2d_transpose_38 (Conv2DT (None, 8, 8, 2)           65538     
_________________________________________________________________
dropout_28 (Dropout)         (None, 8, 8, 2)           0         
_________________________________________________________________
conv2d_transpose_39 (Conv2DT (None, 16, 16, 2)         1026      
_________________________________________________________________
dropout_29 (Dropout)         (None, 16, 16, 2)         0         
_________________________________________________________________
conv2d_transpose_40 (Conv2DT (None, 32, 32, 2)         4098      
_________________________________________________________________
dropout_30 (Dropout)         (None, 32, 32, 2)       

In [ ]:
model_with_resnet.fit(
    X_train, new_Y,
    validation_split=0.1,
    batch_size = batch_size,
    epochs = 300
)

In [7]:
from tensorflow.keras.applications import InceptionV3

#Model parameters
stride = 32
learning_rate = 0.1
N_CLASSES = 2
batch_size = 2
N_epochs = 30

# Init the InceptionV3 model
efnet_conv = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
for layer in efnet_conv.layers[:]:
    layer.trainable = False

# ENCODER
model_with_efnet = Sequential()
model_with_efnet.add(efnet_conv)


# Upsampling

model_with_efnet.add(Deconvolution2D(N_CLASSES, (16, 16), strides=(4, 4), padding='same') )
model_with_efnet.add(Dropout(0.2))

model_with_efnet.add(Deconvolution2D(N_CLASSES, (32, 32), strides=(4, 4), padding='same') )
model_with_efnet.add(Dropout(0.2))

model_with_efnet.add(Deconvolution2D(N_CLASSES, (32, 32), strides=(2, 2), padding='same') )
model_with_efnet.add(Dropout(0.2))


model_with_efnet.add(Deconvolution2D(N_CLASSES, (64, 64), strides=(2, 2), padding='same') )



model_with_efnet.add(Activation('softmax'))
model_with_efnet.summary()        

m = tf.keras.metrics.MeanIoU(num_classes=2)

model_with_efnet.compile(loss="binary_crossentropy",
              optimizer="Adam",
              metrics=['accuracy', m, dice ])

87916544/87910968 [==============================] - 1s 0us/step
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 2, 2, 2048)        21802784  
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 8, 8, 2)           1048578   
_________________________________________________________________
dropout_4 (Dropout)          (None, 8, 8, 2)           0         
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 32, 32, 2)         4098      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 32, 2)         0         
_________________________________________________________________
conv2d_transpose_8 (Conv2DTr (None, 64, 64, 2)         4098      
_______________________________________________________

In [ ]:
model_with_efnet.fit(
    X_train, new_Y,
    validation_split=0.1,
    batch_size = batch_size,
    epochs = 300
)